# Семинар 7: продвинутые функции

### Рекурсия

Пусть мы хотим написать функцию быстрого возведения числа в степень:

```
def pow(base: Union[int, float], deg: int)
```

Известно, что:
```
pow(a, n)
    = pow(a, n // 2) ** 2, если n -- четное
    = a * pow(a, n // 2) ** 2, если n -- нечетное
```

In [1]:
# n - четное: a^n = (a^(n / 2))^2
# n - нечетное: a^n = a^(n - 1 + 1) = a * a^(n - 1)

# n = 0: pow(a, n) = 1
# a = 0: pow(a, n) = 0

In [3]:
from typing import Union


def pow(base: Union[int, float], deg: int):
    if base == 0:
        return 0  # а без этого получим DivisionByZeroError, если deg < 0 [1]
    if deg == 0:
        return 1  # без этого получим бесконечное выполнение

    # a^(-n) = (1/a)^n
    elif deg < 0:
        deg *= -1
        base = 1 / base  # [1] вот тут

    if not deg % 2:
        return pow(base, deg // 2) ** 2
    return base * pow(base, deg // 2) ** 2


In [4]:
pow(2, 2)

4

Что тут произошло? Правильно, мы вызвали из функции саму себя, получается так называемая рекурсия.

С рекурсией стоит быть аккуратным: неаккуратное ее использование приведет к бесконечной работе программы, а также она выжрет у вас всю память компа или питон прибьет ее раньше.

Под капотом рекурсия работает как стэк вызовов. Каждый новый вызов помещается на его верхушку, после выполнения работы функции на этапе рекурсии этот вызов со стэка снимается. Проще всего это заметить если Сделать искусственный пример, где выдадим ошибку в какой-то момент времени.

In [5]:
def recursive(n: int):
    if n == 0:
        raise RuntimeError("Just checking call stack")

    recursive(n - 1)


In [6]:
recursive(6)

RuntimeError: ignored

### Проблема лимита рекурсии

Пусть мы хотим рекурсивно сумму для всех чисел от 1 до n (примечание: ДА, я знаю, что это можно просто сделать циклом, это игрушечный пример)

In [8]:
def recursive(n: int):
    if n == 1:
        return 1

    return n + recursive(n - 1)

In [9]:
recursive(3)

6

In [10]:
recursive(10000)  # ой, а че это мы такое поймали?

RecursionError: ignored

In [11]:
import sys

sys.getrecursionlimit()  # по умолчанию в питоне очень маленький лимит на рекурсивные вызовы

1000

In [12]:
sys.setrecursionlimit(10 ** 6)  # надо быть аккуратным

recursive(10000)

50005000

### Проблема отсутствия мемоизации

Пусть мы хотим посчитать $n$-ое число Фибоначчи

$F_1 = F_2 = 1$

$F_n = F_{n - 1} + F_{n - 2}$

In [14]:
def fib(n: int):
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)  # рекурсивно берем предыдущие числа Фибоначчи

А теперь давайте подумаем: сколько будет работать такой код?

In [15]:
%%time
fib(8)

CPU times: user 15 µs, sys: 2 µs, total: 17 µs
Wall time: 20.3 µs


21

In [16]:
%%time
fib(34)  # тут уже будет долго

CPU times: user 1.45 s, sys: 733 µs, total: 1.45 s
Wall time: 1.46 s


5702887

In [17]:
%%time
fib(40)  # и чем дальше, тем хуже, а 40 -- это же еще не то чтобы много

CPU times: user 28.7 s, sys: 46.8 ms, total: 28.7 s
Wall time: 29.4 s


102334155

In [20]:
%%time
n = 40
a, b = 1, 1
for i in range(3, n + 1):
    a, b = a + b, a
print(a)

102334155
CPU times: user 516 µs, sys: 11 µs, total: 527 µs
Wall time: 535 µs


Проблема заключается в том, что у нас есть дубликаты вызовов. Поэтому питон делает много дополнительных действий. Чем дальше по номеру число Фибоначчи -- тем больше придется решать одинаковых абсолютно вызовов рекурсии. Решить это можно мемоизацией:

In [29]:
cache = dict()

def fib(n: int) -> int:
    if n <= 2:
        return 1
    if n in cache:
        return cache[n]
    else:
        cache[n] = fib(n - 1) + fib(n - 2)
        return cache[n]

In [30]:
%%time
fib(40)  # быстро? быстро!

CPU times: user 67 µs, sys: 0 ns, total: 67 µs
Wall time: 71.8 µs


102334155

In [34]:
from collections import defaultdict


SENTINEL = object()


fib_cache = defaultdict(lambda: SENTINEL)


def set_cache(n: int, result: int):
    fib_cache[n] = result


def fib(n: int):
    if fib_cache[n] is not SENTINEL:
        return fib_cache[n]

    if n <= 2:
        set_cache(n, 1)
        return 1

    result = fib(n - 1) + fib(n - 2)
    set_cache(n, result)
    return result

In [35]:
%%time
fib(40)  # быстро? быстро!

CPU times: user 80 µs, sys: 2 µs, total: 82 µs
Wall time: 86.3 µs


102334155

In [36]:
%%time
fib(1000)  # быстро? быстро!

CPU times: user 2.21 ms, sys: 5 µs, total: 2.22 ms
Wall time: 2.24 ms


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

Очевидно, что это уже получается громоздко и неудобно, поэтому кэширование в питоне есть в том числе и встроенное.

Есть так называемые декораторы `@cache` и `@lru_cache` (least recently used). Второй выкидывает из кэша элементы, когда его размер достиг заданного максимума. При этом `@cache` устроен вот так:

```(python)
def cache(user_function, /):
    return lru_cache(maxsize=None)(user_function)
```

У `@lru_cache` параметр maxsize по умолчанию равен 128, можно увеличить при желании.

In [39]:
from functools import cache, lru_cache

# lru_cache -- можно еще указать параметр maxsize
@lru_cache(maxsize=1000)  # <--- это декоратор, про них поговорим чуть позже
def fib(n: int):
    if n <= 2:
        return 1

    return fib(n - 1) + fib(n - 2)

In [40]:
%%time
fib(1000)

CPU times: user 2.6 ms, sys: 0 ns, total: 2.6 ms
Wall time: 2.66 ms


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

### Что еще умеет functools?

Есть какие-то очень специфичные функции, например, [cmp_to_key](https://docs.python.org/3/library/functools.html#functools.cmp_to_key), который не очень рекомендуется использовать без особой надобности.

Но есть и полезные штуки:

In [42]:
from functools import partial

my_list = list(map(int, input().split()))

4 12 2 3 6 1 23 5 3


In [44]:
get_sorted_list = partial(sorted, my_list)  # создаем функцию из функции прямо в коде = sorted(my_list)

In [47]:
get_sorted_list(reverse=True)  # sorted(my_list, reverse=True)

[23, 12, 6, 5, 4, 3, 3, 2, 1]

In [48]:
get_reverse_sorted_list = partial(sorted, my_list, reverse=True)  # = sorted(my_list, reversed=True)

In [50]:
get_reverse_sorted_list(key=lambda x: -x)

[1, 2, 3, 3, 4, 5, 6, 12, 23]

In [52]:
def add(a, b):
    return a + b

# def add10(a):
#     return a + 10

add10 = partial(add, 10)
add10(19)

29

Можно заранее и не передавать объект:

In [53]:
from typing import Callable


def reverse_number(n: int) -> int:
    return int(str(n)[::-1])  # 456 -> 654

def sort_digits(n: int) -> int:
    return int("".join(sorted(str(n))))  # 672 -> 267

def add_one(n: int) -> int:
    return n + 1

def combine(*functions: Callable[[int], int], number: int) -> int:
    result = number
    for func in functions:
        result = func(result)
    return result

# по сути partial позволяет заполнить какие-то аргументы заранее, а остальные дозаполнить потом
do_some_stuff = partial(combine, add_one, reverse_number, add_one, add_one, sort_digits)
do_another_stuff = partial(combine, reverse_number, add_one, add_one, add_one)

In [54]:
do_some_stuff(number=456)

567

In [55]:
do_another_stuff(number=456)

657

## Декораторы (optional)

In [85]:
import time

def fancy_function():
    time.sleep(1)
    print("I'm a fancy function")

In [86]:
start = time.time()
fancy_function()
end = time.time()
print(end - start)

I'm a fancy function
1.0020358562469482


In [87]:
def wrap_with_timer(func):
    def inner():
        start = time.time()
        func()
        end = time.time()
        print(end - start)
    return inner

In [90]:
# fancy_function = wrap_with_timer(fancy_function)

@wrap_with_timer
def fancy_function():
    time.sleep(1)
    print("I'm a fancy function")

In [91]:
fancy_function()

I'm a fancy function
1.0019216537475586


In [93]:
def wrap_with_timer(func):
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(end - start)
        return result
    return inner

@wrap_with_timer
def fancy_function(a, b, c):
    time.sleep(1)
    print("I'm a fancy function")
    return a + b + c

In [94]:
fancy_function(1, 2, 3)

I'm a fancy function
1.0012321472167969


6

In [95]:
def fancy_function(a, b, c):
    time.sleep(1)
    print("I'm a fancy function")
    return a + b + c

In [96]:
fancy_function.__name__

'fancy_function'

In [97]:
def wrap_with_timer(func):
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(end - start)
        return result
    return inner

# fancy_function = wrap_with_timer(fancy_function)
@wrap_with_timer
def fancy_function(a, b, c):
    time.sleep(1)
    print("I'm a fancy function")
    return a + b + c

In [98]:
fancy_function.__name__

'inner'

In [103]:
from functools import wraps

def wrap_with_timer(func):
    @wraps(func)
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(end - start)
        return result
    return inner

# fancy_function = wrap_with_timer(fancy_function)
@wrap_with_timer
def fancy_function(a, b, c):
    time.sleep(1)
    print("I'm a fancy function")
    return a + b + c

In [104]:
fancy_function.__name__

'fancy_function'

In [105]:
# def inner(func):
#     start = time.time()
#     func()
#     end = time.time()
#     print(end - start)

# def wrap_with_timer(func):
#     inner_ = partial(inner, func)
#     return inner_

# def fancy_function():
#     time.sleep(1)
#     print("I'm a fancy function")

# fancy_function = wrap_with_timer(fancy_function)
# fancy_function()

# Практика

### Задача 1
Пусть у нас есть словарь любой вложенности. При этом на самом нижнем уровне у него всегда находится строка. Ключи тоже всегда строки. Надо написать функцию, которая принимает на себя такой словарь, и выводит на экран все строки на нижнем уровне, в любом порядке.

```(python)
a = {
    "x": {
        "y": "you_killed_kenny",
        "w": "south_park",
    },
    "p": "matt_stone",
}

print(recursive_print(a))
```

Для того, чтобы понять, что перед нами -- словарь или список -- можно воспользоваться функцией `isinstance(some_variable, dict)` -- она вернет True, если это словарь и False иначе

In [106]:
def recursive_print(a):
    if isinstance(a, str):
        print(a)
    elif isinstance(a, dict):
        for value in a.values():
            recursive_print(value)
    else:
        raise ValueError("something strange")

In [107]:
a = {
    "x": {
        "y": "you_killed_kenny",
        "w": "south_park",
    },
    "p": "matt_stone",
}

recursive_print(a)

you_killed_kenny
south_park
matt_stone


### Задача 2

Напишите программу, которая выбирает их полученной на вход последовательности (каждое число на новой строке) квадраты целых чисел выводит их в обратном порядке. Использовать массив для хранения последовательности не разрешается.

### Задача 3

Для быстрого вычисления наибольшего общего делителя двух чисел используют алгоритм Евклида. Он построен на следующем соотношении: $gcd(a, b) = gcd(b, a \mod b)$

Реализуйте рекурсивный алгоритм Евклида в виде функции gcd(a, b).